In [ ]:
import json
def merge_json_files(file_list):
    merged_data = []
    for file in file_list:
        with open(file, 'r') as f:
            data = json.load(f)
            merged_data += data
    return merged_data
 
files = ["./data/train/qwen/qwen_train_DR.json", "./data/train/prm800k/prm_800k_orm_A_DR.json"]
merged_json = merge_json_files(files)

goal_name = "./data/train/qwen/mix_DR.json"
with open(goal_name, 'w') as json_file:
    json.dump(merged_json, json_file, indent=2)

In [5]:
import copy
import json

def read_json(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data

def keep_same_num_merge(prm_path, qwen_path):
    prm_datas = read_json(prm_path)
    qwen_datas = read_json(qwen_path)

    print(len(prm_datas))
    print(len(qwen_datas))
    print(len(prm_datas) + len(qwen_datas))
    qwen_questions = dict()
    for data in qwen_datas:
        question = data["messages"][0]["content"]
        if question not in qwen_questions.keys():
            qwen_questions[question] = 0
        qwen_questions[question] += 1
    
    new_datas = copy.deepcopy(qwen_datas)
    for data in prm_datas:
        question = data["messages"][0]["content"]
        if question in qwen_questions.keys():
            qwen_questions[question] -= 1
            if qwen_questions[question] >= 0:
                continue
        new_datas.append(data)
    print(len(new_datas))
    return new_datas

prm_path = "../train/prm800k/prm_800k_orm_A.json" #"./data/train/prm800k/prm_800k_orm_A.json"
qwen_path = "../train/qwen/qwen_train_DR.json" #"./data/train/qwen/qwen_train_DR.json"
merged_json = keep_same_num_merge(prm_path, qwen_path)
goal_name = "../train/qwen/fair_mix_DR.json" #"./data/train/qwen/fair_mix_DR.json"
with open(goal_name, 'w') as json_file:
    json.dump(merged_json, json_file, indent=2)

98322
12060
110382
99231


In [55]:
import pandas as pd
import json

json_file_path = "../special tokens.json"
with open(json_file_path, 'r') as file:
    data = json.load(file)
pos_key = "postive"
neg_key = "negtive"
nat_key = "natural"

def load_question(path):
    questions = []
    answers = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            entry = json.loads(line)
            question = entry["question"]["problem"]
            answer = entry["question"]["ground_truth_answer"]
            if question not in questions:
                questions.append(question)
                answers.append(answer)
    return questions, answers

def handle_format(question, solution, label, generation_fromat):
    if label:
        step_val = pos_key
    else:
        step_val = neg_key
    if generation_fromat:
        outs = {
            "messages": [
                {
                "content": question,
                "role": "user"
                },
                {
                "content": solution,
                "role": "assistant"
                },
                {
                "content": "Is the answer correct (Yes/No)?",
                "role": "user"
                },
                {
                "content": "Yes" if label else "No", #simplify selection score
                "role": "assistant"
                }
            ],
            "label": label,
            "step_val":step_val
        }
    else:
        outs = {
            "messages": [
                {
                "content": question,
                "role": "user"
                },
                {
                "content": solution,
                "role": "assistant"
                }
            ],
            "label": label,
            "step_val":step_val
        }
    return  outs

df = pd.read_parquet('train-00000-of-00001.parquet')
questions = df["problem"]
question_set = set(list(questions))
train_question, _ = load_question("../prm_800k_handler/data/phase2_train.jsonl")
question_set = [x for x in questions if x not in train_question]
max_length = 1024
N_size = 8
generation_format = False
goal_name = "../train/qwen/BON_D_test.json" 

write_datas = []
for row in df.itertuples():
    tques = row.problem
    if tques in question_set:
        question_set.remove(tques)
    else:
        continue
    labels = row.scores
    responses = []
    for i in range(32):
        name = "response_"+str(i+1)
        res = getattr(row, name)
        if len(res) >= max_length:
            continue
        responses.append(res)
    if len(responses) < N_size:
        continue
    for i in range(N_size):
        write_datas.append(handle_format(tques, responses[i], bool(labels[i]), True))

print(len(write_datas))
with open(goal_name, 'w') as json_file:
    json.dump(write_datas, json_file, indent=2)


312
